### Q-Value

**Q-Value** is a measure of the overall expected reward assuming the agent is in state $s$ and performs action $a$, and then continues playing until the end of the episode following some policy $\pi$. It is defined mathematically as:

\begin{equation}
Q^{\pi}\left(s_{t}, a_{t}\right)=E\left[R_{t+1}+\gamma R_{t+2}+\gamma^{2} R_{t+3}+\ldots | s_{t}, a_{t}\right]
\end{equation}

where $R_{t+1}$ is the immediate reward received after performing action $a_{t}$ in state $s_{t}$ and $\gamma$ is the discount factor and controls the importance of the future rewards versus the immediate ones: the lower the discount factor is, the less important future rewards are.

### Optimal Q-Value

To calculate the optimal **Q-Value** for a given **state-action pair**,  $Q^{*}(s,a)$, we use the **Bellman Optimality Equation**.

### Bellman Optimality Equation

The Bellman equation defines the relationships between a given **state-action pair** and its successors. While many forms exist, one of the most common is the **Bellman Optimality Equation** which gives us the optimal **Q-Value**, which is given by:

$$
Q^{*}(s, a)=\sum_{s^{\prime}, r} p\left(s^{\prime}, r | s, a\right)\left[r+\gamma \max _{a^{\prime}} Q^{*}\left(s^{\prime}, a^{\prime}\right)\right]
$$

Of course, when no uncertainty exists (transition probabilities are either 0 or 1), we have:

$$
Q^{*}(s, a)=r(s, a)+\gamma \max _{a^{\prime}} Q^{*}\left(s^{\prime}, a^{\prime}\right)
$$

### Q-Value Iteration

We define the corresponding Bellman backup operator:

$$
[\mathcal{T} Q]\left(s, a\right)=r(s, a)+\gamma \max _{a^{\prime}} Q\left(s^{\prime}, a^{\prime}\right)
$$

This means that when $Q$ is a fixed point of $\mathcal{T}$ the following becomes true:

$$
\mathcal{T} Q^{*}=Q^{*}
$$

So if we apply the Bellman operator $\mathcal{T}$ repeatedly to any initial $Q$, the series converges to $Q^{*}$:

$$
Q, \mathcal{T} Q, \mathcal{T}^{2} Q, \cdots \rightarrow Q^{*}
$$

## DQN

Deep Q Network (DQN) was proposed by researchers at [Deepmind](https://deepmind.com/) in a paper in 2015: [*Human-level control through deep reinforcement learning*](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) (Minh et al 2015).

Whilst this was not the first paper to propose using neural networks to approximate the action-value function, it was the first to demonstrate that the same neural network architecture could be trained in a computationally efficient manner to "solve" a large number of different tasks.

### Problems
Prior to this paper it was well known that standard reinforcement learning algorithms were unstable or even diverged when non-linear function approximators such as a neural networks were used to represent the action-value function $Q$. The main problems at the time where:

1. Correlations present in the sequence of observations of the state $s$. In reinforcement learning applications the sequence state observations is a time-series which will almost surely be auto-correlated. But surely this would also be true of any application of deep neural networks to model time series data. 
2. Small updates to $Q$ may significantly change the policy, $\pi$ and therefore change the data distribution.
3. Correlations between the action-values, $Q$, and the target values $r + \gamma \max_{a'} Q(s', a')$

### Solutions
The paper's authors addressed these issues by proposing the following solutions:

* a biologically inspired mechanism referred to as [*Experience Replay*](#experience_replay) that randomly selects previous experience to replay from the data. By doing this it removes correlations in the sequence of observations of the state $s$ and smoothes over changes in the data distribution (issues 1 and 2 above).
* an iterative update rule known as [*Q-learning Update*](#q_learning_update) that adjusts the action-values, $Q$, towards target values, $Q'$ that are only periodically updated thereby reducing correlations with the target (issue 3 above).

<a id='experience_replay'></a>
### Experience Replay

To perform *experience replay* the authors store the agent's experiences $e_t$ as represented by the tuple

$$ e_t = (s_t, a_t, r_t, s_{t+1}) $$

consisting of the observed state in period $t$, the reward received in period $t$, the action taken in period $t$, and the resulting state in period $t+1$. The dataset of agent experiences at period $t$ consists of the set of past experiences.

$$ D_t = \{e1, e2, ..., e_t \} $$

Depending on the task it may note be feasible for the agent to store the entire history of past experiences.

During learning Q-learning updates are computed based on samples (or minibatches) of experience $(s,a,r,s')$, drawn uniformly at random from the pool of stored samples $D_t$.

<a id='q_learning_update'></a>
### Q-learning Update
Another way to write the Q-learning Bellman equation is to separate *action selection* from *action evaluation*:

$$ Y_t^{DQN} = r_{t} + \gamma Q\big(s_{t+1}, \underset{a}{\mathrm{argmax}}\ Q(s_{t+1}, a; \theta_t); \theta_t\big) $$

### DQN Loss Function

We get an experience tuple $(s,a,r,s')$ by sampling uniformly randomly from the replay buffer $D$. This is denoted as $U(D)$ in the equation below.

The $Q$-learning update at iteration $i$ uses the following loss function:

$$ \mathcal{L_i}(\theta_i) = \mathbb{E}_{(s, a, r, s') \sim U(D)} \Bigg[\bigg(r + \gamma Q\big(s', \underset{a'}{\mathrm{argmax}}\ Q(s',a'; \theta^{-}\big);\theta^{-}\bigg) - Q\big(s, a; \theta_{i}\big)\bigg)\Bigg] $$

Where $\gamma$ is the discount factor determining the agent’s horizon, $\theta_i$ are the parameters of the $Q$-network at iteration $i$ and $\theta_i^{-}$ are the $Q$-network parameters used to compute the target at iteration $i$. The target network parameters $\theta_i^{-}$ are only updated with the $Q$-network parameters $\theta_i$ every $C$ steps and are frozen between individual updates. 

## Double DQN
The Double DQN algorithm is described in the paper [*Deep Reinforcement Learning with Double Q-Learning*](https://arxiv.org/abs/1509.06461) (Van Hasselt et al 2015). The Double DQN algorithm is a minor, but important, modification of the original DQN algorithm.

### Problems
* Q-learning can be overly optimistic at the large-scale, even to the point of being deterministic, which in turn leads to problems due to the inherent estimation errors of learning. 
* Overestimations are more common and severe in practice than previously had been acknowledged.
* The presence of the max operator in the Bellman equation used to compute the $Q$-values means that the approximate $Q$-values will almost always be strictly greater than the corresponding $Q$ values from the true action-value function (i.e., the approximation errors will almost always be positive). This potentially significant source of bias can impede learning and is often exacerbated by the use of flexible, non-linear function approximators such as neural networks.

### Solutions
The Van Hasselt et al 2015 paper makes several important contributions. 

* Proposed the implementation of a Double Q-learning method called "Double DQN" that extends, with minor modifications, the original DQN algorithm
* Double DQN can be used at scale to successfully reduce overestimations. This results in more stable and reliable learning.
* Double DQN finds better policies by obtaining new state-of-the-art results on the Atari 2600 dataset.
* Double DQN explicitly separates action selection from action evaluation which allows each step to use a different function approximator resulting in a better overall approximation of the action-value function.

## Implementing the Double DQN algorithm

The key idea behind Double Q-learning is to reduce overestimations of Q-values by separating the selection of actions from the evaluation of those actions so that a different Q-network can be used in each step.

### Double DQN Q-learning Update
To implement the Double DQN algorithm a second action-value function is introduced and used with the greedy policy actions before the Q-network estimate is calculated. We extend the original DQN Q-learning update equation shown above by using the online Q-network $Q(S, a; \theta)$ to select the actions and then use the target Q-network $Q(S, a; \theta^{-})$ to evaluate the selected actions. The Q-learning update for Double DQN then becomes:

$$ Y_t^{DoubleDQN} = r_{t} + \gamma Q\big(s_{t+1}, \underset{a}{\mathrm{argmax}}\ Q(s_{t+1}, a; \theta_t), \theta_t^{-}\big) $$

### DDQN Loss Function

We get an experience tuple $(s,a,r,s')$ by sampling uniformly randomly from the replay buffer $D$. This is denoted as $U(D)$ in the equation below.

The $Q$-learning update at iteration $i$ uses the following loss function:

$$ \mathcal{L_i}(\theta_i) = \mathbb{E}_{(s, a, r, s') \sim U(D)} \Bigg[\bigg(r + \gamma Q\big(s', \underset{a'}{\mathrm{argmax}}\ Q(s',a'; \theta_{i}\big);\theta^{-}\bigg) - Q\big(s, a; \theta_i\big)\bigg)\Bigg] $$

where $\gamma$ is the discount factor determining the agent’s horizon, $\theta_i$ are the parameters of the $Q$-network at iteration $i$ and $\theta_i^{-}$ are the $Q$-network parameters used to compute the target at iteration $i$. The target network parameters $\theta_i^{-}$ are only updated with the $Q$-network parameters $\theta_i$ every $C$ steps and are frozen between individual updates. We use the online weights to select the action but still use the frozen weights to get the estimate.